In [1]:
import numpy as np
import healpy as hp

In [2]:
def speedy_solver(detsvals,detnorms,bottheta,toptheta,botphi,topphi,n,background):
   # mask2 = np.array(detsvals) >background
    #detsvals = detsvals[mask2]
    theta = np.deg2rad(np.linspace(bottheta,toptheta,n))
    phi = np.deg2rad(np.linspace(botphi,topphi,n))
    mtheta,mphi = np.meshgrid(theta,phi)
    chiveco = hp.ang2vec(mtheta,mphi)
    chivecs = np.concatenate(chiveco)
    As= np.linspace(10,1000,30)
    chiterms = np.zeros(len(As)*len(theta)*len(phi))
   # print("Len chi terms: the zeros one.. " + str(len(chiterms)))
    for s in range(len(detsvals)):
        normarr = detnorms[s]
   #    print("normal array " + str(s) + " " + str(np.rad2deg(hp.vec2ang(normarr)))) 
        normarrs = []
        for garc in range((len(theta)*len(phi))):
            normarrs.append([normarr[0],normarr[1],normarr[2]])
        
        seps = findAngles(chivecs,normarrs)
        
        #mask = seps < np.pi/2
        #seps[mask]
        #only using seps under 90, good. 
        AA,SS = np.meshgrid(As,seps)

        Aofit = np.concatenate(AA)
        chiseps = np.concatenate(SS)
        bg = background * np.ones(len(chiseps))
        #when chiseps in here are >90, make response huuuge
        good = chiseps < np.pi/2
        bad = chiseps > np.pi/2
        chiseps[good]
        chiResponse = np.multiply(Aofit,response(chisepschiseps[good])) + bg
         
    #  print("iteration : " + str(s))
        chiterm = np.divide(np.power(np.subtract(chiResponse,detsvals[s]),2),detsvals[s])
      #  print("chiterm: " + str(chiterm))

        chiterms += chiterm
     #   print("chiterms final: " + str(chiterms))
    chimin = min(chiterms)
    chisquareds = list(chiterms)

    thetaloc = np.rad2deg(theta[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(phi)*len(Aofit))))/(len(phi)*len(Aofit)))])
    philoc = np.rad2deg(phi[int(((chisquareds.index(chimin) % (len(phi)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(phi)*len(Aofit)))  % len(Aofit))]
    
    return thetaloc,philoc,Aoguess
